In [1]:
import azureml.core
from azureml.core import Workspace

ws = Workspace.from_config()

# Train and Register the Model

In [22]:
from azureml.core import Experiment
from azureml.core import Model
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

experiment = Experiment(workspace = ws, name = 'mslearn-train-diabetes')
run = experiment.start_logging()

print("Starting the Experiment: ", experiment.name)

# Loading the diabetes dataset
diabetes = pd.read_csv('data/diabetes.csv')

# Creating the model
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# Metrics
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# Saving the model in outputs folder
model_file = 'diabetes_model.pkl'
joblib.dump(value=model, filename=model_file)
run.upload_file(name = 'outputs/' + model_file, path_or_stream = './' + model_file)

run.complete()

# Registering the model
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Inline Training'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})



Starting the Experiment:  mslearn-train-diabetes
Training a decision tree model
Accuracy: 0.8933333333333333
AUC: 0.8791283148415343


Model(workspace=Workspace.create(name='aml-workspace', subscription_id='3571f8dc-3527-4993-9d2b-ac0812d807fd', resource_group='aml-resources'), name=diabetes_model, id=diabetes_model:2, version=2, tags={'Training context': 'Inline Training'}, properties={'AUC': '0.8791283148415343', 'Accuracy': '0.8933333333333333'})

# Deploying the model as a webservice

In [23]:
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, "version: ", model.version)

    print('Model Tags: ')
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print('\t', tag_name, ':', tag)
    
    print('Model Properties:')
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print('\t', prop_name, ':', prop)

    print('\n')


diabetes_model version:  2
Model Tags: 
	 Training context : Inline Training
Model Properties:
	 AUC : 0.8791283148415343
	 Accuracy : 0.8933333333333333


diabetes_model version:  1
Model Tags: 
	 Training context : Inline Training
Model Properties:
	 AUC : 0.8781009005423155
	 Accuracy : 0.891




**If we mention model name without the version, by default, latest version will be retrived.**

In [24]:
model = ws.models['diabetes_model']
print(model.name, 'version: ', model.version)

diabetes_model version:  2


**Creating a folder to keep everything at once place**

In [25]:
import os

deployment_folder = './diabetes_service'
os.makedirs(deployment_folder, exist_ok = True)
print(deployment_folder, ' : Folder is created.')

script_file = 'score_diabetes.py'
script_path = os.path.join(deployment_folder, script_file)

./diabetes_service  : Folder is created.


In [27]:
%%writefile $script_path
import json
import joblib
import numpy as np
import os

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'diabetes_model.pkl')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = np.array(json.loads(raw_data)['data'])
    # Get a prediction from the model
    predictions = model.predict(data)
    # Get the corresponding classname for each prediction (0 or 1)
    classnames = ['not-diabetic', 'diabetic']
    predicted_classes = []
    for prediction in predictions:
        predicted_classes.append(classnames[prediction])
    # Return the predictions as JSON
    return json.dumps(predicted_classes)

Writing ./diabetes_service/score_diabetes.py


# Deploying the Model

In [28]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

# Configure the scoring environment
serive_env = Environment.get(workspace = ws, name = 'AzureML-sklearn-0.24.1-ubuntu18.04-py37-cpu-inference')
serive_env.inferencing_stack_version = 'latest'

inference_config = InferenceConfig(source_directory = deployment_folder, entry_script = script_file, environment = serive_env)

# Configure the webservice container
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 2, memory_gb = 1)

# Deploy Model as a service
print('Deploying Model...')
service_name = 'diabetes-service'
service  = Model.deploy(ws, service_name, [model], inference_config, deployment_config, overwrite = True)
service.wait_for_deployment(True)

print(service.state)

Deploying Model...
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-08-26 07:24:04+00:00 Creating Container Registry if not exists..
2022-08-26 07:34:05+00:00 Registering the environment.
2022-08-26 07:34:09+00:00 Generating deployment configuration..
2022-08-26 07:34:12+00:00 Submitting deployment to compute..
2022-08-26 07:34:21+00:00 Checking the status of deployment diabetes-service..
2022-08-26 07:35:24+00:00 Checking the status of inference endpoint diabetes-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


**Healthy status indicates deployment is successful**

In [29]:
print(service.get_logs)

<bound method Webservice.get_logs of AciWebservice(workspace=Workspace.create(name='aml-workspace', subscription_id='3571f8dc-3527-4993-9d2b-ac0812d807fd', resource_group='aml-resources'), name=diabetes-service, image_id=None, image_digest=None, compute_type=ACI, state=Healthy, scoring_uri=http://603e1ddd-2364-4121-9892-08067d8f875c.centralindia.azurecontainer.io/score, tags={}, properties={'hasInferenceSchema': 'False', 'hasHttps': 'False', 'authEnabled': 'False'}, created_by={'userObjectId': '3264cdde-0bb3-4d5c-b3e2-50d81926d518', 'userPuId': '1003200217F96528', 'userIdp': 'live.com', 'userAltSecId': '1:live.com:00030000DF07121D', 'userIss': 'https://sts.windows.net/78c76086-2fb7-4f6a-b684-c129ba0ea713/', 'userTenantId': '78c76086-2fb7-4f6a-b684-c129ba0ea713', 'userName': 'Neeresh Kumar Perla', 'upn': None})>


**To print all webseries in our workspace**

In [30]:
for webservice in ws.webservices:
    print(webservice)

diabetes-service


# Scoring new data using the webservice (Inferencing)

In [31]:
import json

x_new = [[2,180,74,24,21,23.9091702,1.488172308,22]]
print ('Patient: {}'.format(x_new[0]))

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data (the web service will also accept the data in binary format)
predictions = service.run(input_data = input_json)

# Get the predicted class - it'll be the first (and only) one.
predicted_classes = json.loads(predictions)
print(predicted_classes[0])

Patient: [2, 180, 74, 24, 21, 23.9091702, 1.488172308, 22]
diabetic


In [32]:
import json

# This time our input is an array of two feature arrays
x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

# Convert the array or arrays to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data
predictions = service.run(input_data = input_json)

# Get the predicted classes.
predicted_classes = json.loads(predictions)
   
for i in range(len(x_new)):
    print ("Patient {}".format(x_new[i]), predicted_classes[i] )

Patient [2, 180, 74, 24, 21, 23.9091702, 1.488172308, 22] diabetic
Patient [0, 148, 58, 11, 179, 39.19207553, 0.160829008, 45] not-diabetic


**To this url applications submit their request**

In [33]:
endpoint = service.scoring_uri
print(endpoint)

http://603e1ddd-2364-4121-9892-08067d8f875c.centralindia.azurecontainer.io/score


In [34]:
import requests
import json

x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Set the content type
headers = { 'Content-Type':'application/json' }

predictions = requests.post(endpoint, input_json, headers = headers)
predicted_classes = json.loads(predictions.json())

for i in range(len(x_new)):
    print ("Patient {}".format(x_new[i]), predicted_classes[i] )

Patient [2, 180, 74, 24, 21, 23.9091702, 1.488172308, 22] diabetic
Patient [0, 148, 58, 11, 179, 39.19207553, 0.160829008, 45] not-diabetic


# Deleting the service

In [35]:
service.delete()
print ('Service deleted.')

Service deleted.


In [36]:
for webservice in ws.webservices:
    print(webservice)

**We deleted the deployment service sucessfully...!**